## VTK Clip with Bounds
This example is inspired from this ``pyvista`` example:
https://docs.pyvista.org/examples/01-filter/clipping.html#clip-with-bounds

In [ ]:
import pyvista as pv
import panel as pn

import pyvista.examples as examples
pn.extension('vtk')

In [ ]:
dataset = examples.download_office()
p = pv.Plotter()
initial_camera = {'position': [13.078393697738647, 7.325431823730469, 7.7784422636032104],
                  'focalPoint': [2.5192227363586426, 2.29958438873291, 0.9933322267606854],
                  'viewUp': [-0.3366442620754242, -0.4262550473213196, 0.8396292328834534],}

wire = p.add_mesh(dataset, style='wireframe', color='blue')

vtkpan = pn.panel(p.ren_win, height=500, width=500)
vtkpan.reset_camera()

x_clip = pn.widgets.RangeSlider(value=(2,4.5), value_throttled=(2,4.5), 
                                start=0, end=4.5, step=0.1, name="x_clip")
y_clip = pn.widgets.RangeSlider(value=(2,4.5), value_throttled=(2,4.5),
                                start=0, end=4.5, step=0.1, name="y_clip")
z_clip = pn.widgets.RangeSlider(value=(1,3), value_throttled=(1,3),
                                start=0, end=3, step=0.1, name="z_clip")
opacity = pn.widgets.FloatSlider(value=1, value_throttled=1,
                                 start=0, end=1, step=0.1, name='opacity') 
wbox = pn.Column(pn.Row(x_clip, y_clip, z_clip, width=500), opacity)

clipped_actor = None
@pn.depends(x_clip=x_clip.param.value_throttled,
            y_clip=y_clip.param.value_throttled,
            z_clip=z_clip.param.value_throttled)
def clip_mesh(x_clip, y_clip, z_clip):
    global clipped_actor
    bounds = [x_clip[0], x_clip[1],
              y_clip[0], y_clip[1],
              z_clip[0], z_clip[1]]
    clipped = dataset.clip_box(bounds)
    if clipped_actor is None:
        clipped_actor = p.add_mesh(clipped, label='Clipped')
    else:
        clipped_actor.GetMapper().SetInputData(clipped)
    clip_opactity(opacity.value)

@pn.depends(opacity=opacity.param.value_throttled)
def clip_opactity(opacity):
    global clipped_actor
    if clipped_actor:
        clipped_actor.GetProperty().SetOpacity(opacity)
        vtkpan.synchronize()

#creation the equivalent volume to compare:
import vtk
probe_filter = vtk.vtkProbeFilter()
xdim, ydim, zdim = 51, 51, 51
xextent, yextent, zextent = 4.5, 4.5, 3.

unif_grid = pv.UniformGrid((xdim, ydim, zdim), #dim 
                           (xextent/xdim, yextent/ydim, zextent/zdim), #spacing
                           origin=(0,0,0))
probe_filter.SetInputData(unif_grid)
probe_filter.SetSourceData(dataset)
probe_filter.Update()
probe_filter.Update()
vtkvol = pn.panel(probe_filter.GetOutput(), render_background='#4c4c4c', height=500, width=500,
                  camera=initial_camera)
vtkpan.jslink(vtkvol, camera='camera', bidirectional=True)

pn.Column(pn.Row(wbox), pn.Row(vtkpan, vtkvol), clip_mesh, clip_opactity, background='#eeeeee').servable()